In [3]:
from collections import defaultdict
from multiprocessing import Pool

from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from consent.consistency.cookie_pref_match import cookie_pref_match
from consent.consistency.util import get_scan_root_dir, get_scan_dirs # , FIG_DIR uncomment to save fig.
from consent.data.pref_menu_scan.cookie_pref_reader import read_cookie_prefs_in_scans
from consent.data.pref_menu_scan.log_reader import read_logs_in_scans
from consent.data.pref_menu_scan.cookie_decl_reader import read_cookie_decls_in_scans
from consent.data.pref_menu_scan.har_cookie_reader import read_postrej_sent_cookies_in_scans
from ooutil.type_util import hashabledict

SCAN_DIRS = get_scan_dirs('eu')
SCAN_ROOT_DIR = get_scan_root_dir('eu')

In [4]:
# Get cookie prefs = cookie decls + prefs
cookie_prefs = read_cookie_prefs_in_scans(SCAN_DIRS)  # took 2.3min for top 50k-site scan
cookie_prefs.head() 

Error reading intel.co.uk dictionary update sequence element #0 has length 1; 2 is required
Fail to read log 1 sites: ['intel.co.uk'] 
https://cdn.cookielaw.org/consent/55ba86fb-59dd-478c-99c8-7cb04534ceb6/a1db9992-7dcf-4e4b-849b-484b65d4fc8a/en.json WARNING: Cookie list has session cookie but duration > 0
https://cdn.cookielaw.org/consent/c570eb43-1169-4222-914c-38a09bd70ca0/a1f914ac-390b-484b-9577-7c7ad52c8361/en.json WARNING: Cookie list has session cookie but duration > 0
https://cdn-ukwest.onetrust.com/consent/e00c3e10-0e07-461a-94d5-e87b899fa55a/442b5be0-8f10-4555-b6a6-2aa7c2e23dfa/en.json WARNING: Cookie list has session cookie but duration > 0
https://cdn.cookielaw.org/consent/be922b85-1c88-4393-ae73-1523f02934d6/366288bf-75c8-472c-b77b-7724367eecb9/en.json WARNING: Cookie list has session cookie but duration > 0
https://cdn.cookielaw.org/consent/bfc3f6d1-88f6-4292-b5d2-757633a696fe/c9582860-fa9f-44f1-b41d-4d91b7388022/en.json WARNING: Cookie list has session cookie but duratio

,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent
0,__we_bucket_id,www.wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True
1,OptanonAlertBoxClosed,wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True
2,OptanonConsent,www.wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True
3,__we_request_id,www.wework.com,Session,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True
4,we_referring_domain,www.wework.com,14 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True


In [5]:
all_complies_file = SCAN_ROOT_DIR / 'all_complies.parquet'
all_complies = pd.read_parquet(all_complies_file)
all_complies

,domain,expires,name,path,sameSite,secure,value,request_url,site,comply
0,.wework.com,1.671518e+09,ajs_user_id,/,Lax,False,%22b22ada9465f804d5ec485afec1845a7c8078ec584f5...,https://www.wework.com/vanilla-assets/javascri...,wework.com,incorrect
1,www.wework.com,-1.000000e+00,__we_request_id,/,Lax,True,0b5ee6f3-64b9-4002-91c9-5c657dcd2634,https://www.wework.com/vanilla-assets/javascri...,wework.com,comply
2,www.wework.com,1.671518e+09,__we_bucket_id,/,Lax,True,b22ada9465f804d5ec485afec1845a7c8078ec584f59b9...,https://www.wework.com/vanilla-assets/javascri...,wework.com,comply
3,.wework.com,1.671518e+09,ajs_user_id,/,Lax,False,%22b22ada9465f804d5ec485afec1845a7c8078ec584f5...,https://www.wework.com/vanilla-assets/javascri...,wework.com,incorrect
4,.wework.com,1.671518e+09,ajs_anonymous_id,/,Lax,False,%22ef8417d2-09a6-41af-be22-e46a6e0ea92d%22,https://www-static.wework.com/apercu/apercu.css,wework.com,incorrect
...,...,...,...,...,...,...,...,...,...,...
2952203,www.sentinelone.com,1.703098e+09,driftt_aid,/,None,True,2908eb81-db4a-45a6-bb9b-30b9c7da1695,https://www.sentinelone.com/wp-content/themes/...,sentinelone.com,omit
2952204,www.sentinelone.com,1.640027e+09,_omappvs,/,Lax,True,1640026120556,https://www.sentinelone.com/wp-content/themes/...,sentinelone.com,omit
2952205,.sentinelone.com,1.671562e+09,OptanonConsent,/,Lax,False,isGpcEnabled=0&datestamp=Mon+Dec+20+2021+18%3A...,https://www.sentinelone.com/wp-content/themes/...,sentinelone.com,comply
2952206,.sentinelone.com,1.703098e+09,_ga,/,None,False,GA1.2.1830428201.1640026025,https://www.sentinelone.com/wp-content/themes/...,sentinelone.com,incorrect


In [6]:
site_to_libname = {row['site']: row['lib_name'] for row in cookie_prefs[['site', 'lib_name']].drop_duplicates().to_dict('records')}
from collections import Counter
libs = [site_to_libname[site] for site in all_complies.site.unique()]
Counter(libs)

Counter({'onetrust': 1705, 'cookiebot': 220})